In [ ]:
#bigbasket dynamic products by scrolling

In [48]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.edge.options import Options
from bs4 import BeautifulSoup
import time
import csv
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Connection': 'keep-alive',
    'DNT': '1',
    'Upgrade-Insecure-Requests': '1'
    }

In [49]:
driver = webdriver.Edge()
wait = WebDriverWait(driver, 10)


def search_category():
    cat_in = input("").strip().lower()
    cat_ab = (
        "apparel","fruits-vegetables", "foodgrains-oil-masala", "bakery-cakes-dairy", "beverages",
        "snacks-branded-foods", "beauty-hygiene", "cleaning-household", "kitchen-garden-pets",
        "lunch-boxes-bags", "umbrellas-rainwear","eggs-meat-fish", "gourmet-world-food", "baby-care", "paan-corner"
    )
    for k in cat_ab:
        for j in k.split("-"):
            if cat_in in j:
                return k
    return None

def scrap_category(search):
    if not search:
        return []

    url = f"https://www.bigbasket.com/cl/{search}/?nc=nb"
    driver.get(url)
    # time.sleep(2)
    product_links = set()  
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True :
        driver.execute_script("window.scrollBy(0, window.innerHeight*1.5);")
        time.sleep(3)  # Adjust the sleep time if needed
        
        # Check if the page height has increased
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # Exit if no new content is loaded
        last_height = new_height
        
    # Extract product links from the current view
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    all_cards = soup.find_all('div', class_="SKUDeck___StyledDiv-sc-1e5d9gk-0 eA-dmzP")
    for card in all_cards:
        link = card.find('a', rel="noopener noreferrer")
        if link:
            product_links.add(f"https://www.bigbasket.com{link.get('href')}")
            
    return list(product_links)


def scrap_product(url_page):
    try:
        driver.get(url_page)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Extract brand, title, category, price, weight/volume, and images
        brand = soup.find('a', class_="Description___StyledLink-sc-82a36a-1 gePjxR")
        title = soup.find('h1', class_="Description___StyledH-sc-82a36a-2 bofYPK")
        category_elements = soup.find_all('span', class_="name text-md leading-md xl:leading-sm xl:text-base text-darkOnyx-800 false")
        price = soup.find('td', class_="Description___StyledTd-sc-82a36a-4 fLZywG")
        w_v = soup.find('span', class_="Label-sc-15v1nk5-0 PackSizeSelector___StyledLabel2-sc-l9rhbt-2 gJxZPQ hDJUsF")

        # Process extracted elements
        brand_text = brand.text.strip() if brand else None
        title_text = title.text.strip() if title else None
        price_text = price.text[8:].strip() if price else None
        w_v_text = w_v.text.strip() if w_v else None

        # Construct category path
        category = " -> ".join([elem.text for elem in category_elements]) if category_elements else None

        # Determine count, quantity, and unit from weight/volume text
        count = 1
        unit = ""
        if not w_v_text and title_text:
            sample = title_text.split()
            units = ["kg", "ml", "pc"]
            prev = ""
            for unit in units:
                for word in sample:
                    if unit in word:
                        w_v_text = prev + word
                        unit = word.strip()
                        break
                    prev = word
        
        if w_v_text:
            w_v_text = w_v_text.strip()
            unit = w_v_text[-2:].strip()
            if "x" in w_v_text:
                w_v_parts = w_v_text.split("x")
                count = w_v_parts[0].strip()
                if "pc" in w_v_parts[1]:
                    w_v_text = w_v_parts[1][:-3].strip()
                    unit = "pcs"
                else:
                    numbers = re.findall(r'\d+', w_v_parts[1])
                    alphabets = re.findall(r'[a-zA-Z]+', w_v_parts[1])
                    w_v_text = ' '.join(numbers)
                    unit = ' '.join(alphabets)
            else:
                if " " in w_v_text:
                    parts = w_v_text.split()
                    w_v_text = parts[0]
                    unit = parts[1]
                else:
                    numbers = re.findall(r'\d+', w_v_text)
                    alphabets = re.findall(r'[a-zA-Z]+', w_v_text)
                    w_v_text = ' '.join(numbers)
                    unit = ' '.join(alphabets)

        # Extract image links
        image_links = []
        images = soup.find_all('img', lazyboundary="800px")
        for img in images[1::2]:  # Ensure correct slicing
            link = img.get('src')
            if link:
                link = link.replace("?tr=w-256,q=80", "")
                link = link.replace("/s/", "/l/")
                image_links.append(link)

        # Extract "About the Product" and "Other Product Info" sections
        details = {}
        about_section = soup.find('span', string='About the Product')
        other_info_section = soup.find('span', string='Other Product Info')

        if about_section:
            about_content = about_section.find_next("div").find_next("div")
            details["About the Product"] = about_content.text.strip() if about_content else None

        if other_info_section:
            other_info_content = other_info_section.find_next("div").find_next("div")
            details["Other Product Info"] = other_info_content.text.strip() if other_info_content else None
            if "EAN" in details.get("Other Product Info", ""):
                ean_matches = re.findall(r'\d+', details["Other Product Info"])
                details["EAN Code"] = ean_matches[0] if ean_matches else None

        # Construct the product dictionary
        product = {
            "Brand": brand_text,
            "Title": title_text,
            "Category": category,
            "Price": price_text,
            "Count": count,
            "Quantity": w_v_text,
            "Unit": unit,
            "Image Link": image_links,
            "Url":url_page
        }

        product.update(details)
        return product

    except Exception as e:
        print(f"Error: {e} while processing {url_page}")
        return None
    

# Function to handle multithreading
def scrap_product_concurrently(product_links):
    product_details = []
    with ThreadPoolExecutor(max_workers=2) as executor:  # Adjust max_workers as needed
        future_to_url = {executor.submit(scrap_product, url): url for url in product_links}
        for future in as_completed(future_to_url):
            url = future_to_url[future]
            try:
                product = future.result()
                if product:
                    product_details.append(product)
            except Exception as exc:
                print(f'{url} generated an exception: {exc}')
    return product_details

# Main function
if __name__ == '__main__':
    category = search_category()
    print(category)
    product_detail = []
    if category:
        product_links = scrap_category(category)
        print(f"Found {len(product_links)} products.")
        if product_links:
            product_detail = scrap_product_concurrently(product_links)
    else:
        print("No category found")

    if product_detail:
        keys = product_detail[0].keys()
        with open('bigbasket_data.csv', 'a', newline='', encoding='utf-8') as output_file:
            dict_writer = csv.DictWriter(output_file, fieldnames=keys)
            # dict_writer.writeheader()  # Uncomment if writing the header for the first time
            dict_writer.writerows(product_detail)
    else:
        print("No product found")

    print("Processing done")

driver.quit()

 umbrella


umbrellas-rainwear
Found 141 products.
Processing done


In [32]:
# beverages-959
# apparel-76
# umbrella-141
# fruits-vegetables-434


In [ ]:
## Main function
# if __name__ == '__main__':
#     category = search_category()
#     print(category)
#     product_detail = []
#     if category:
#         product_links = scrap_category(category)
#         print(f"Found {len(product_links)} products.")
#         for link in product_links:
#             product = scrap_product(link)
#             if product:
#                 product_detail.append(product)
#     else:
#         print("No category found")

#     if len(product_detail) > 0:
#         # print(product_detail)
#         keys = product_detail[0].keys()
#         with open('bigbasket_data.csv', 'a', newline='', encoding='utf-8') as output_file:
#             dict_writer = csv.DictWriter(output_file, fieldnames=keys)
#             # dict_writer.writeheader()
#             dict_writer.writerows(product_detail)
#     else:
#         print("No product found")

#     print("Processing done")

# driver.quit()

In [ ]:
#flipkart grocery

In [41]:
driver = webdriver.Edge()
wait = WebDriverWait(driver, 2)

def search_category():
    category_onsite={"staples-dals-pulses":"https://www.flipkart.com/grocery/staples/dals-pulses/pr?sid=73z,bpe,3uv&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_45413f33-7ee3-4e87-85a2-b53a6a20b73b_2_G211C67CJ4GB_MC.PAR0XYGYH8FZ&otracker=dynamic_rich_navigation_1_2.navigationCard.RICH_NAVIGATION_Staples~Dals%2B%2526%2BPulses_PAR0XYGYH8FZ&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_1_L1_view-all&cid=PAR0XYGYH8FZ",
                 "staples-ghee-oils":"https://www.flipkart.com/grocery/staples/ghee-oils/pr?sid=73z,bpe,4wu&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_e71133a4-33d6-4a3f-9f19-51e01c12fbb2_2_G211C67CJ4GB_MC.QW9A7WPEBLP6&otracker=dynamic_rich_navigation_2_2.navigationCard.RICH_NAVIGATION_Staples~Ghee%2B%2526%2BOils_QW9A7WPEBLP6&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_2_L1_view-all&cid=QW9A7WPEBLP6",
                 "staples-atta-flours":"hhttps://www.flipkart.com/grocery/snacks-beverages/tea/pr?sid=73z,ujs,amr&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_4cb3a283-7077-404d-a6c0-ecb06702e646_2_G211C67CJ4GB_MC.WGRAZWGDPE1W&otracker=dynamic_rich_navigation_3_2.navigationCard.RICH_NAVIGATION_Snacks%2B%2526%2BBeverages~Tea_WGRAZWGDPE1W&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_3_L1_view-all&cid=WGRAZWGDPE1Wttps://www.flipkart.com/grocery/staples/atta-flours/pr?sid=73z,bpe,9da&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_138b11fa-9a79-432c-a132-15fc83c64915_2_G211C67CJ4GB_MC.ZWBFCX62G9LP&otracker=dynamic_rich_navigation_3_2.navigationCard.RICH_NAVIGATION_Staples~Atta%2B%2526%2BFlours_ZWBFCX62G9LP&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_3_L1_view-all&cid=ZWBFCX62G9LP",
                 "staples-masalas-spices":"https://www.flipkart.com/grocery/staples/masalas-spices/pr?sid=73z,bpe,a6m&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_90f0e2e9-95c6-45b1-8b05-9ff97e6d2b6a_2_G211C67CJ4GB_MC.QFQ3Q4E8ZB2I&otracker=dynamic_rich_navigation_4_2.navigationCard.RICH_NAVIGATION_Staples~Masalas%2B%2526%2BSpices_QFQ3Q4E8ZB2I&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_4_L1_view-all&cid=QFQ3Q4E8ZB2I",
                 "staples-rice-rice-products":"https://www.flipkart.com/grocery/staples/rice-rice-products/pr?sid=73z,bpe,zwp&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_69204c4a-fefb-4b7e-be6d-f10bb5f5ebb0_2_G211C67CJ4GB_MC.GQEJYG9UMBCO&otracker=dynamic_rich_navigation_5_2.navigationCard.RICH_NAVIGATION_Staples~Rice%2B%2526%2BRice%2BProducts_GQEJYG9UMBCO&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_5_L1_view-all&cid=GQEJYG9UMBCO",
                 "staples-dry-fruits-nuts-seeds":"https://www.flipkart.com/grocery/staples/dry-fruits-nuts-seeds/pr?sid=73z,bpe,dtp&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_27470102-4a85-478a-93fd-6e015ed092ee_2_G211C67CJ4GB_MC.X697907L5MB0&otracker=dynamic_rich_navigation_6_2.navigationCard.RICH_NAVIGATION_Staples~Dry%2BFruits%252C%2BNuts%2B%2526%2BSeeds_X697907L5MB0&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_6_L1_view-all&cid=X697907L5MB0",
                 "staples-sugar-jaggery-salt":"https://www.flipkart.com/grocery/staples/sugar-jaggery-salt/pr?sid=73z,bpe,fdl&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_66ba0670-7db5-4c4c-a7f9-0e5706af9808_2_G211C67CJ4GB_MC.XI6TQHBVLUGF&otracker=dynamic_rich_navigation_7_2.navigationCard.RICH_NAVIGATION_Staples~Sugar%252C%2BJaggery%2B%2526%2BSalt_XI6TQHBVLUGF&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_7_L1_view-all&cid=XI6TQHBVLUGF",
                 "snacks-beverages-biscuits":"https://www.flipkart.com/grocery/snacks-beverages/biscuits/pr?sid=73z,ujs,eb9&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_6e04215a-3399-4124-823a-f00543adbb02_2_G211C67CJ4GB_MC.AZ9TH6KGXIQX&otracker=dynamic_rich_navigation_1_2.navigationCard.RICH_NAVIGATION_Snacks%2B%2526%2BBeverages~Biscuits_AZ9TH6KGXIQX&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_1_L1_view-all&cid=AZ9TH6KGXIQX",
                 "snacks-beverages-chipsnamkeen-snacks":"https://www.flipkart.com/grocery/snacks-beverages/chipsnamkeen-snacks/pr?sid=73z,ujs,dd9&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_60a46e9e-74c5-4eae-a26f-fdb447e4f44a_2_G211C67CJ4GB_MC.ZTW4LDXW99L4&otracker=dynamic_rich_navigation_2_2.navigationCard.RICH_NAVIGATION_Snacks%2B%2526%2BBeverages~Chips%252CNamkeen%2B%2526%2BSnacks_ZTW4LDXW99L4&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_2_L1_view-all&cid=ZTW4LDXW99L4",
                 "snacks-beverages-tea":"https://www.flipkart.com/grocery/snacks-beverages/tea/pr?sid=73z,ujs,amr&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_4cb3a283-7077-404d-a6c0-ecb06702e646_2_G211C67CJ4GB_MC.WGRAZWGDPE1W&otracker=dynamic_rich_navigation_3_2.navigationCard.RICH_NAVIGATION_Snacks%2B%2526%2BBeverages~Tea_WGRAZWGDPE1W&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_3_L1_view-all&cid=WGRAZWGDPE1W",
                 "snacks-beverages-coffee":"https://www.flipkart.com/grocery/snacks-beverages/coffee/pr?sid=73z,ujs,t7k&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_d95bbc9a-f963-41a5-baf0-a36bc206b1b8_2_G211C67CJ4GB_MC.098FWDWWR6ZV&otracker=dynamic_rich_navigation_4_2.navigationCard.RICH_NAVIGATION_Snacks%2B%2526%2BBeverages~Coffee_098FWDWWR6ZV&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_4_L1_view-all&cid=098FWDWWR6ZV",
                 "snacks-beverages-juices":"https://www.flipkart.com/grocery/snacks-beverages/juices/pr?sid=73z,ujs,afd&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_6d4535aa-2144-43dd-9ffe-f75ef7c3ef06_2_G211C67CJ4GB_MC.W5PUVK1VCI95&otracker=dynamic_rich_navigation_5_2.navigationCard.RICH_NAVIGATION_Snacks%2B%2526%2BBeverages~Juices_W5PUVK1VCI95&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_5_L1_view-all&cid=W5PUVK1VCI95",
                 "snacks-beverages-nutrition-health-drink-mix":"https://www.flipkart.com/grocery/snacks-beverages/health-drink-mix/pr?sid=73z,ujs,vnq&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_a35c9936-e890-4da1-a27b-610b88ed847f_2_G211C67CJ4GB_MC.0UDCKV2OVSBR&otracker=dynamic_rich_navigation_6_2.navigationCard.RICH_NAVIGATION_Snacks%2B%2526%2BBeverages~Nutritional%2BDrink%2BMix_0UDCKV2OVSBR&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_6_L1_view-all&cid=0UDCKV2OVSBR",
                 "snacks-beverages-soft-drinks":"https://www.flipkart.com/grocery/snacks-beverages/soft-drinks/pr?sid=73z,ujs,dfw&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_4ff7704f-4130-4f92-8eae-4247d40eb2f4_2_G211C67CJ4GB_MC.L6I5OTOPKRJ0&otracker=dynamic_rich_navigation_7_2.navigationCard.RICH_NAVIGATION_Snacks%2B%2526%2BBeverages~Soft%2BDrinks_L6I5OTOPKRJ0&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_7_L1_view-all&cid=L6I5OTOPKRJ0",
                 "snacks-beverages-instant-drink-mix-squash-syrups":"https://www.flipkart.com/grocery/snacks-beverages/squash-syrups/pr?sid=73z,ujs,iau&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_202b9653-99b1-4f4b-a216-0f9397850bbf_2_G211C67CJ4GB_MC.X92O1TC1W20T&otracker=dynamic_rich_navigation_8_2.navigationCard.RICH_NAVIGATION_Snacks%2B%2526%2BBeverages~Instant%2BDrink%2BMixes%252C%2BSquash%2B%2526%2BSyrups_X92O1TC1W20T&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_8_L1_view-all&cid=X92O1TC1W20T",
                 "breakfast-cereals":"https://www.flipkart.com/grocery/packaged-food/breakfast-cereals/pr?sid=73z,u0u,bx9&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_42445ea2-9b58-4bc3-8e29-c1b77a9cf468_2_G211C67CJ4GB_MC.NXLSWOZVU3Q0&otracker=dynamic_rich_navigation_1_2.navigationCard.RICH_NAVIGATION_Packaged%2BFood~Breakfast%2BCereals_NXLSWOZVU3Q0&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_1_L1_view-all&cid=NXLSWOZVU3Q0",
                 "packaged-food-noodles-pasta":"https://www.flipkart.com/grocery/packaged-food/noodles-pasta/pr?sid=73z,u0u,ltz&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_3e15933a-6639-4327-9eec-8214786c8500_2_G211C67CJ4GB_MC.GOSA97N625VX&otracker=dynamic_rich_navigation_2_2.navigationCard.RICH_NAVIGATION_Packaged%2BFood~Noodles%2B%2526%2BPasta_GOSA97N625VX&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_2_L1_view-all&cid=GOSA97N625VX",
                 "packaged-food-ketchups-spreads":"https://www.flipkart.com/grocery/packaged-food/ketchups-spreads/pr?sid=73z,u0u,0tl&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_ef1e300b-07a3-4f62-964f-872b81b88df0_2_G211C67CJ4GB_MC.Y2HU0NO6WEWM&otracker=dynamic_rich_navigation_3_2.navigationCard.RICH_NAVIGATION_Packaged%2BFood~Ketchups%2B%2526%2BSpreads_Y2HU0NO6WEWM&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_3_L1_view-all&cid=Y2HU0NO6WEWM",
                 "packaged-food-chocolates-sweets":"https://www.flipkart.com/grocery/packaged-food/chocolates-sweets/pr?sid=73z,u0u,7o6&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_6f63635d-617c-468e-882e-b8cd33b64296_2_G211C67CJ4GB_MC.XDTF6QJ4BWBW&otracker=dynamic_rich_navigation_4_2.navigationCard.RICH_NAVIGATION_Packaged%2BFood~Chocolates%2B%2526%2BSweets_XDTF6QJ4BWBW&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_4_L1_view-all&cid=XDTF6QJ4BWBW",
                 "packaged-food-jams-honey":"https://www.flipkart.com/grocery/packaged-food/jams-honey/pr?sid=73z,u0u,j4e&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_60e2fbad-6ed8-4e13-94c6-b2b59afb9d7f_2_G211C67CJ4GB_MC.NOMWVMJOO0PM&otracker=dynamic_rich_navigation_5_2.navigationCard.RICH_NAVIGATION_Packaged%2BFood~Jams%2B%2526%2BHoney_NOMWVMJOO0PM&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_5_L1_view-all&cid=NOMWVMJOO0PM",
                 "packaged-food-pickles-chutney":"https://www.flipkart.com/grocery/packaged-food/pickles-chutney/pr?sid=73z,u0u,03x&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_b0875f57-5364-469c-b4f3-1ce6306b1f0f_2_G211C67CJ4GB_MC.SYSHNZJX28YU&otracker=dynamic_rich_navigation_6_2.navigationCard.RICH_NAVIGATION_Packaged%2BFood~Pickles%2B%2526%2BChutney_SYSHNZJX28YU&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_6_L1_view-all&cid=SYSHNZJX28YU",
                 "packaged-food-ready-to-cook":"https://www.flipkart.com/grocery/packaged-food/ready-to-cook/pr?sid=73z,u0u,0gv&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_dce33539-3b47-4308-b8d5-24239d3db8c9_2_G211C67CJ4GB_MC.N8BWXHWL92JS&otracker=dynamic_rich_navigation_7_2.navigationCard.RICH_NAVIGATION_Packaged%2BFood~Ready%2BTo%2BCook_N8BWXHWL92JS&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_7_L1_view-all&cid=N8BWXHWL92JS",
                 "packaged-food-cooking-sauces-vinegar":"https://www.flipkart.com/grocery/packaged-food/cooking-sauces-vinegar/pr?sid=73z,u0u,wd7&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_52612f35-ddd1-4f69-87a9-b4643c807e10_2_G211C67CJ4GB_MC.VV2FEO6GLPZD&otracker=dynamic_rich_navigation_8_2.navigationCard.RICH_NAVIGATION_Packaged%2BFood~Cooking%2BSauces%2B%2526%2BVinegar_VV2FEO6GLPZD&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_8_L1_view-all&cid=VV2FEO6GLPZD",
                 "packaged-food-baking":"https://www.flipkart.com/grocery/packaged-food/baking/pr?sid=73z,u0u,td1&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_9d9073ff-4228-46bd-bf20-fb22209a9b0d_2_G211C67CJ4GB_MC.BL6HT50QLUZU&otracker=dynamic_rich_navigation_9_2.navigationCard.RICH_NAVIGATION_Packaged%2BFood~Baking_BL6HT50QLUZU&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_9_L1_view-all&cid=BL6HT50QLUZU",
                 "personal-baby-care-soaps-body-wash":"https://www.flipkart.com/grocery/personal-baby-care/soaps-body-wash/pr?sid=73z,njl,sn6&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_f11a68d6-be37-48c3-8863-e86b30185690_2_G211C67CJ4GB_MC.7F0DETGHROTO&otracker=dynamic_rich_navigation_1_2.navigationCard.RICH_NAVIGATION_Personal%2B%2526%2B%2BBaby%2BCare~Soaps%2B%2526%2BBody%2BWash_7F0DETGHROTO&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_1_L1_view-all&cid=7F0DETGHROTO",
                 "personal-baby-care-hair-care":"https://www.flipkart.com/grocery/personal-baby-care/hair-care/pr?sid=73z,njl,vpw&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_9334d098-ef54-4b6b-9787-0de1c20f1437_2_G211C67CJ4GB_MC.R7ZRVFUBRXPF&otracker=dynamic_rich_navigation_2_2.navigationCard.RICH_NAVIGATION_Personal%2B%2526%2B%2BBaby%2BCare~Hair%2BCare_R7ZRVFUBRXPF&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_2_L1_view-all&cid=R7ZRVFUBRXPF",
                 "personal-baby-care-oral-care":"https://www.flipkart.com/grocery/personal-baby-care/oral-care/pr?sid=73z,njl,2s3&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_a7ada0e4-b9b1-4aae-846e-43f97ad2f81f_2_G211C67CJ4GB_MC.VW3OLXAN704N&otracker=dynamic_rich_navigation_3_2.navigationCard.RICH_NAVIGATION_Personal%2B%2526%2B%2BBaby%2BCare~Oral%2BCare_VW3OLXAN704N&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_3_L1_view-all&cid=VW3OLXAN704N",
                 "personal-baby-care-deos-perfumes-talc":"https://www.flipkart.com/grocery/personal-baby-care/deos-perfumes-talc/pr?sid=73z,njl,np3&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_91b0de07-e67c-497c-a095-3c29c5b7b23f_2_G211C67CJ4GB_MC.RJC3FQD7ABJ2&otracker=dynamic_rich_navigation_4_2.navigationCard.RICH_NAVIGATION_Personal%2B%2526%2B%2BBaby%2BCare~Deos%252C%2BPerfumes%2B%2526%2BTalc_RJC3FQD7ABJ2&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_4_L1_view-all&cid=RJC3FQD7ABJ2",
                 "personal-baby-care-creams-lotions-skin-care":"https://www.flipkart.com/grocery/personal-baby-care/creams-lotions-skin-care/pr?sid=73z,njl,n3m&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_4279163f-4013-4d88-a706-b7cef7bf9694_2_G211C67CJ4GB_MC.WTZJ5WJG7PHA&otracker=dynamic_rich_navigation_5_2.navigationCard.RICH_NAVIGATION_Personal%2B%2526%2B%2BBaby%2BCare~Creams%252C%2BLotions%252C%2BSkin%2BCare_WTZJ5WJG7PHA&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_5_L1_view-all&cid=WTZJ5WJG7PHA",
                 "personal-baby-care-kajal-makeup":"https://www.flipkart.com/grocery/personal-baby-care/kajal-makeup/pr?sid=73z,njl,lzq&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_feb0a797-962a-4b3b-971f-ff37b9ceb22d_2_G211C67CJ4GB_MC.MZ33JOUJEQ4M&otracker=dynamic_rich_navigation_6_2.navigationCard.RICH_NAVIGATION_Personal%2B%2526%2B%2BBaby%2BCare~Kajal%2B%2526%2BMakeup_MZ33JOUJEQ4M&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_6_L1_view-all&cid=MZ33JOUJEQ4M",
                 "personal-baby-care-sanitary-needs":"https://www.flipkart.com/grocery/personal-baby-care/sanitary-needs/pr?sid=73z,njl,tlj&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_6c1eab08-e116-45dc-a035-b6eb25b675ed_2_G211C67CJ4GB_MC.Z0TGVHD7I91A&otracker=dynamic_rich_navigation_7_2.navigationCard.RICH_NAVIGATION_Personal%2B%2526%2B%2BBaby%2BCare~Sanitary%2BNeeds_Z0TGVHD7I91A&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_7_L1_view-all&cid=Z0TGVHD7I91A",
                 "personal-baby-care-wellness-common-pharma":"https://www.flipkart.com/grocery/personal-baby-care/wellness-common-pharma/pr?sid=73z,njl,07d&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_544e3db6-9ad7-44f2-8963-b3e40a8dc543_2_G211C67CJ4GB_MC.2Y1DUHDVM8UF&otracker=dynamic_rich_navigation_8_2.navigationCard.RICH_NAVIGATION_Personal%2B%2526%2B%2BBaby%2BCare~Wellness%2B%2526%2BCommon%2BPharma_2Y1DUHDVM8UF&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_8_L1_view-all&cid=2Y1DUHDVM8UF",
                 "personal-baby-care-shaving-needs":"https://www.flipkart.com/grocery/personal-baby-care/shaving-needs/pr?sid=73z,njl,nw3&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_f92ec933-099c-4b2b-b391-49006c5f73ab_2_G211C67CJ4GB_MC.I7MGHRBTU853&otracker=dynamic_rich_navigation_9_2.navigationCard.RICH_NAVIGATION_Personal%2B%2526%2B%2BBaby%2BCare~Shaving%2BNeeds_I7MGHRBTU853&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_9_L1_view-all&cid=I7MGHRBTU853",
                 "personal-baby-care-diapers-wipes":"https://www.flipkart.com/grocery/personal-baby-care/diapers-wipes/pr?sid=73z,njl,smb&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_147a2113-3030-4e97-a1e3-0d9ede3da40e_2_G211C67CJ4GB_MC.UGJV25JFCWSR&otracker=dynamic_rich_navigation_10_2.navigationCard.RICH_NAVIGATION_Personal%2B%2526%2B%2BBaby%2BCare~Diapers%2B%2526%2BWipes_UGJV25JFCWSR&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_10_L1_view-all&cid=UGJV25JFCWSR",
                 "personal-baby-care-baby-foods":"https://www.flipkart.com/grocery/personal-baby-care/baby-foods/pr?sid=73z,njl,2jj&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_fc471c26-4e83-4c1f-af56-c4912f0f2879_2_G211C67CJ4GB_MC.5HNYYGGUGK4A&otracker=dynamic_rich_navigation_11_2.navigationCard.RICH_NAVIGATION_Personal%2B%2526%2B%2BBaby%2BCare~Baby%2BFoods_5HNYYGGUGK4A&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_11_L1_view-all&cid=5HNYYGGUGK4A",
                 "personal-baby-care-baby-bath-skin-care":"https://www.flipkart.com/grocery/personal-baby-care/baby-bath-skin-care/pr?sid=73z,njl,m1f&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_51283b8e-e897-4aa5-8ef0-9a33ec49d98e_2_G211C67CJ4GB_MC.KS27G12L39SU&otracker=dynamic_rich_navigation_12_2.navigationCard.RICH_NAVIGATION_Personal%2B%2526%2B%2BBaby%2BCare~Baby%2BBath%2B%2526%2BSkin%2BCare_KS27G12L39SU&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_12_L1_view-all&cid=KS27G12L39SU",
                 "household-care-detergents-laundry":"https://www.flipkart.com/grocery/household-care/detergents-laundry/pr?sid=73z,cwl,2z2&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_7468389a-8736-46d9-bbb1-78998f101429_2_G211C67CJ4GB_MC.8NV4Z62U9QS3&otracker=dynamic_rich_navigation_1_2.navigationCard.RICH_NAVIGATION_Household%2BCare~Detergents%2B%2526%2BLaundry_8NV4Z62U9QS3&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_1_L1_view-all&cid=8NV4Z62U9QS3",
                 "household-care-utensil-cleaners":"https://www.flipkart.com/grocery/household-care/utensil-cleaners/pr?sid=73z,cwl,bdc&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_ebebca08-9b27-4a8b-a6ea-35ba2825dcd7_2_G211C67CJ4GB_MC.WBQVNZTUYITO&otracker=dynamic_rich_navigation_2_2.navigationCard.RICH_NAVIGATION_Household%2BCare~Utensil%2BCleaners_WBQVNZTUYITO&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_2_L1_view-all&cid=WBQVNZTUYITO",
                 "household-care-floor-other-cleaners":"https://www.flipkart.com/grocery/household-care/floor-other-cleaners/pr?sid=73z,cwl,u3c&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_a44c73f2-14c5-4131-805b-155f088b4562_2_G211C67CJ4GB_MC.L8H68E76OXNJ&otracker=dynamic_rich_navigation_3_2.navigationCard.RICH_NAVIGATION_Household%2BCare~Floor%2B%2526%2BOther%2BCleaners_L8H68E76OXNJ&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_3_L1_view-all&cid=L8H68E76OXNJ",
                 "household-care-repellants-fresheners":"https://www.flipkart.com/grocery/household-care/repellants-fresheners/pr?sid=73z,cwl,qz9&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_464ca411-58f5-4f34-92a4-b9a7bfc91d3a_2_G211C67CJ4GB_MC.BHLX1BPW8BFY&otracker=dynamic_rich_navigation_4_2.navigationCard.RICH_NAVIGATION_Household%2BCare~Repellants%2B%2526%2BFresheners_BHLX1BPW8BFY&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_4_L1_view-all&cid=BHLX1BPW8BFY",
                 "household-care-paper-disposables":"https://www.flipkart.com/grocery/household-care/paper-disposables/pr?sid=73z,cwl,2wc&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_4c84514c-923c-4361-a12a-9d06bb08c268_2_G211C67CJ4GB_MC.1JRSLDRSO719&otracker=dynamic_rich_navigation_5_2.navigationCard.RICH_NAVIGATION_Household%2BCare~Paper%2B%2526%2BDisposables_1JRSLDRSO719&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_5_L1_view-all&cid=1JRSLDRSO719",
                 "household-care-basic-electricals":"https://www.flipkart.com/grocery/household-care/basic-electricals/pr?sid=73z,cwl,0s4&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_5d45e19a-a992-42ba-abf6-9e4a211f19e9_2_G211C67CJ4GB_MC.HP4FWSHM4NN9&otracker=dynamic_rich_navigation_6_2.navigationCard.RICH_NAVIGATION_Household%2BCare~Basic%2BElectricals_HP4FWSHM4NN9&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_6_L1_view-all&cid=HP4FWSHM4NN9",
                 "household-care-pooja-needs":"https://www.flipkart.com/grocery/household-care/pooja-needs/pr?sid=73z,cwl,u64&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_88d922ec-7540-4963-9bd1-a3672974a98d_2_G211C67CJ4GB_MC.2L6HN2TIPCK6&otracker=dynamic_rich_navigation_7_2.navigationCard.RICH_NAVIGATION_Household%2BCare~Pooja%2BNeeds_2L6HN2TIPCK6&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_7_L1_view-all&cid=2L6HN2TIPCK6",
                 "household-care-pet-food":"https://www.flipkart.com/grocery/household-care/pet-food/pr?sid=73z,cwl,m92&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_ef2e1a6d-e7c3-4b9e-8d7a-931b5b9e1e5a_2_G211C67CJ4GB_MC.Y9ZFDE0JI0AM&otracker=dynamic_rich_navigation_8_2.navigationCard.RICH_NAVIGATION_Household%2BCare~Pet%2BFood_Y9ZFDE0JI0AM&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_8_L1_view-all&cid=Y9ZFDE0JI0AM",
                 "household-care-shoe-care":"https://www.flipkart.com/grocery/household-care/shoe-care/pr?sid=73z%2Ccwl%2C5mx&marketplace=GROCERY&pageUID=1579679877380&fm=neo%2Fmerchandising&iid=M_a361b23f-72e4-433e-8ad4-bc3b7a5777b1_2_G211C67CJ4GB_MC.DVUQ5GFL88IF&otracker=dynamic_rich_navigation_9_2.navigationCard.RICH_NAVIGATION_Household%2BCare~Shoe%2BCare_DVUQ5GFL88IF&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_9_L1_view-all&cid=DVUQ5GFL88IF",
                 "dairy":"https://www.flipkart.com/grocery/dairy-eggs/dairy/pr?sid=73z,esa,dt6&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_4af8b4d3-8aa3-403b-9daf-5daea5eb67d1_2_G211C67CJ4GB_MC.O5ZAEV3OQZJF&otracker=dynamic_rich_navigation_1_2.navigationCard.RICH_NAVIGATION_Dairy%2B%2526%2BEggs~Dairy_O5ZAEV3OQZJF&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_1_L1_view-all&cid=O5ZAEV3OQZJF",
                 "eggs":"https://www.flipkart.com/grocery/dairy-eggs/eggs/pr?sid=73z,esa,hh7&otracker=categorytree&marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_a69f684a-2a61-404d-bb72-7479d988443d_2_G211C67CJ4GB_MC.BP8ULPPS8MBO&otracker=dynamic_rich_navigation_2_2.navigationCard.RICH_NAVIGATION_Dairy%2B%2526%2BEggs~Eggs_BP8ULPPS8MBO&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_2_L1_view-all&cid=BP8ULPPS8MBO",
                 "home-kitchen-tools-appliances":"https://www.flipkart.com/grocery/home-kitchen/kitchen-tools-appliances/pr?sid=73z,uyk,hho&marketplace=GROCERY&otracker=product_breadCrumbs_Kitchen+Tools+%26+Appliances&fm=neo%2Fmerchandising&iid=M_67ffdf78-372b-4cfc-b4aa-6eec065435a9_2_G211C67CJ4GB_MC.W18YJL3ZPA2C&otracker=dynamic_rich_navigation_2_2.navigationCard.RICH_NAVIGATION_Home%2B%2526%2BKitchen~Kitchen%2BTools_W18YJL3ZPA2C&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_2_L1_view-all&cid=W18YJL3ZPA2C",
                 "home-kitchen-cookware-non-stick":"https://www.flipkart.com/grocery/home-kitchen/cookware-serveware/pr?sid=73z,uyk,vdx&marketplace=GROCERY&otracker=product_breadCrumbs_Cookware+%26+Serveware&fm=neo%2Fmerchandising&iid=M_b20c856f-1b35-4aeb-8b29-cddd45b8f2d0_2_G211C67CJ4GB_MC.06CU6NJ5NPN2&otracker=dynamic_rich_navigation_3_2.navigationCard.RICH_NAVIGATION_Home%2B%2526%2BKitchen~Cookware%2B%2526%2BNon%2BStick_06CU6NJ5NPN2&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_3_L1_view-all&cid=06CU6NJ5NPN2",
                 "home-kitchen-dining-cutlery":"https://www.flipkart.com/grocery/home-kitchen/cookware-serveware/cutlery-serveware/pr?sid=73z,uyk,vdx,enr&marketplace=GROCERY&otracker=product_breadCrumbs_Cutlery+%26+Serveware&fm=neo%2Fmerchandising&iid=M_9d3ebce5-ab81-48d3-a8de-42de18c0c61a_2_G211C67CJ4GB_MC.TKS76KA8MNTE&otracker=dynamic_rich_navigation_4_2.navigationCard.RICH_NAVIGATION_Home%2B%2526%2BKitchen~Dining%2B%2526%2BCutlery_TKS76KA8MNTE&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_4_L1_view-all&cid=TKS76KA8MNTE",
                 "home-kitchen-stationery-household-school":"https://www.flipkart.com/grocery/office-school-supplies/art-stationery-supplies/pr?sid=73z,pyy,utd&marketplace=GROCERY&otracker=product_breadCrumbs_Art+%26+Stationery+Supplies&fm=neo%2Fmerchandising&iid=M_8861606b-f937-4c42-accb-632bb973171b_2_G211C67CJ4GB_MC.P2GBZRKYKSXT&otracker=dynamic_rich_navigation_5_2.navigationCard.RICH_NAVIGATION_Home%2B%2526%2BKitchen~Stationery%2B%2B%2526%2BHousehold%2BEssentials_P2GBZRKYKSXT&otracker1=dynamic_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_5_L1_view-all&cid=P2GBZRKYKSXT"
                }
    
    cat_in = input("Enter category: ").strip().lower()
    cat_list = list(category_onsite.keys())
    cat_links=[]
    for word in cat_list:
        if cat_in in word:
            cat_links.append(category_onsite[word])
            print(word)
    # print(cat_links)
    return cat_links


def scrap_category(category_links):
    product_links = []
    category_data={}
    a=1
    for url_page in category_links:
        # print("url:",url_page)
        driver.get(url_page)
        
        #pincode
        if a==1:
            pincode_button = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div[1]/div[1]/div[2]/div[3]/div[2]/div/div[2]/div[2]/div/div/div[1]/form/div/button')))
            pincode_button.click()
            time.sleep(2)
            a+=1
            
        while True:
            try:
                # Parse the current page
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                all_cards = soup.find_all('div', class_="M8JyK1")
                for card in all_cards:
                    link = card.find('a', rel="noopener noreferrer").get('href')
                    if link:
                       product_links.append(f"https://www.flipkart.com{link}")
                
                # Find the URL for the next page
                next_button = soup.find('a', class_='_9QVEpD', string="Next")
                if next_button:
                    next_page_url = f"https://www.flipkart.com{next_button.get('href')}"
                    # print(f"Navigating to next page: {next_page_url}")
                    
                    driver.get(next_page_url)
                    time.sleep(2)

            except Exception as e:
                return f"Error: {e}"
                break
    
    return product_links


def scrap_product(url): 

    # driver.get("https://www.flipkart.com/grocery-supermart-store?marketplace=GROCERY&fm=neo%2Fmerchandising&iid=M_c3d27cb6-b179-48d4-96fa-7e33d0311594_1_372UD5BXDFYS_MC.CBUR1Q46W5F1&otracker=hp_rich_navigation_1_1.navigationCard.RICH_NAVIGATION_Grocery_CBUR1Q46W5F1&otracker1=hp_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_1_L0_view-all&cid=CBUR1Q46W5F1")
    # pincode_button = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div[1]/div[1]/div[2]/div[3]/div[2]/div/div[2]/div[2]/div/div/div[1]/form/div/button')))
    # pincode_button.click()
    # time.sleep(2)
    driver.get(url)
    time.sleep(2)
    
    try:
        product_details ,image_links= {},[]
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        #general deatils
        table=soup.find_all('table',class_="_0ZhAN9")[1]
        rows = table.select('table._0ZhAN9 tr.WJdYP6')
        for row in rows:
            # Extract key and value for each row
            key = row.select_one('td.col-3-12').text.strip()
            value = row.select_one('td.col-9-12 li.HPETK2').text.strip()
            product_details[key] = value
        
        image=soup.find_all('img',class_="_0DkuPH")
        for c in image:
            image_links.append(c.get("src"))
        
        title = soup.find('span', class_="VU-ZEz")
        title=title.text if title else None
        
        price=soup.find('div', class_="Nx9bqj CxhGGd")
        price=price.text[1:] if price else None

        description=soup.find('div', class_="yN+eNk w9jEaj")
        description=description.text if description else None

        product_details.update({"title":title,"price":price,"description":description,"Image":image_links})
        keys=["title","Brand","Model Name","price","description","Image","Quantity"]
        
        return dict((k, dictionary[k]) for k in keys if k in product_details)
    except Exception as e:
        print(f"Error: {e}")

if __name__ == '__main__':
    category_links = search_category()
    product_links = scrap_category(category_links)
    print(len(product_links))
    
    for link in product_links:
        print(scrap_product(link))
    
driver.quit()  

Enter category:  dairy


dairy
141
{'Brand': "Cavin's", 'Model Name': 'Masala Chaasa', 'Quantity': '170 ml', 'Key Flavors': 'Masala', 'Container Type': 'Tetrapack', 'Maximum Shelf Life': '181 Days', 'Net Quantity': '170 ml', 'title': "Cavin's Masala Chaasa Buttermilk\xa0\xa0(170 ml)", 'price': None, 'description': None, 'Image': ['https://rukminim2.flixcart.com/image/128/128/l2jcccw0/butter-milk/t/1/y/180-masala-chaasa-tetrapack-1-cavin-s-original-imagdv5fw74gfzwb.jpeg?q=70', 'https://rukminim2.flixcart.com/image/128/128/l2jcccw0/butter-milk/z/y/o/180-masala-chaasa-tetrapack-1-cavin-s-original-imagdv5f2w3hd9jb.jpeg?q=70', 'https://rukminim2.flixcart.com/image/128/128/l2jcccw0/butter-milk/3/p/g/180-masala-chaasa-tetrapack-1-cavin-s-original-imagdv5fwggkwtsd.jpeg?q=70', 'https://rukminim2.flixcart.com/image/128/128/l2jcccw0/butter-milk/v/o/w/180-masala-chaasa-tetrapack-1-cavin-s-original-imagdv5fevxfnfcv.jpeg?q=70']}
{'Brand': "D'lecta", 'Type': 'Mozzarella cheese', 'Quantity': '200 g', 'Organic': 'No', 'Gourmet

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (787, 162)
  (Session info: MicrosoftEdge=126.0.2592.61)
Stacktrace:
	GetHandleVerifier [0x00007FF75DCA8172+13538]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF75DC31E29+595529]
	(No symbol) [0x00007FF75DA4E6CF]
	(No symbol) [0x00007FF75DA98E9E]
	(No symbol) [0x00007FF75DA97101]
	(No symbol) [0x00007FF75DA94E10]
	(No symbol) [0x00007FF75DA93E26]
	(No symbol) [0x00007FF75DA88711]
	(No symbol) [0x00007FF75DAB1FFA]
	(No symbol) [0x00007FF75DA88147]
	(No symbol) [0x00007FF75DA8800D]
	(No symbol) [0x00007FF75DAB22E0]
	(No symbol) [0x00007FF75DA88147]
	(No symbol) [0x00007FF75DACB1EE]
	(No symbol) [0x00007FF75DAB1C63]
	(No symbol) [0x00007FF75DA8766E]
	(No symbol) [0x00007FF75DA8683C]
	(No symbol) [0x00007FF75DA87221]
	Microsoft::Applications::Events::EventProperty::to_string [0x00007FF75DE69714+1099860]
	Microsoft::Applications::Events::EventProperty::~EventPr

In [60]:
url=input()

driver = webdriver.Edge()
wait = WebDriverWait(driver, 2)

product_details ,image_links= {},[]
driver.get("https://www.flipkart.com/grocery-supermart-store?marketplace=GROCERY")

pincode_button = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div/div/div[1]/div[1]/div[2]/div[3]/div[2]/div/div[2]/div[2]/div/div/div[1]/form/div/button')))
pincode_button.click()
time.sleep(2)

soup = BeautifulSoup(driver.page_source, 'html.parser')
image=soup.find_all('img',class_="_0DkuPH")

for c in image:
    image_links.append(c.get("src"))
title = soup.find('span', class_="VU-ZEz").text.strip()  or float("nan")

price=soup.find('div', class_="Nx9bqj CxhGGd")
price=price.text.strip()[1:] if price else None
  
description=soup.find('div', class_="yN+eNk w9jEaj")
description=description.text if description else None
# driver.quit()

print(image_links,title,price,description,sep="\n")

 https://www.flipkart.com/fortune-besan/p/itmetefhhdrhpfge?pid=FLRETEFHZHCFSEDD&lid=LSTFLRETEFHZHCFSEDDWNJ6B1&marketplace=GROCERY&shopId=&spotlightTagId=supercoin_spotlight&iid=f37045fd-de4b-465d-9323-9ddfe98f2e67.FLRETEFHZHCFSEDD.SEARCH


AttributeError: 'NoneType' object has no attribute 'text'